In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from matplotlib.gridspec import GridSpec, GridSpecFromSubplotSpec
import matplotlib.image as mpimg
import pandas as pd
import os
import torch
import torchvision.models as models
import torchvision.transforms as transforms
#from torchvision.io import read_image
from PIL import Image

import pandas as pd

# PATHs
PROJECT_DATA_PATH='//home/bule/projects/MVTec_Visualizer/data/mvtec_anomaly_detection'
LOG_DIR= '/home/bule/projects/MVTec_Visualizer/tensorboard_logs'

In [ ]:
# paths to images
image_paths = []
for root, dirs, files in os.walk(PROJECT_DATA_PATH):
    if 'ground_truth' not in root:
        for file in files:
            if file.endswith('.png'):
                image_paths.append(os.path.join(root, file))
len(image_paths)

In [ ]:
FILENAME='MVTEC_WIDERESNET50_3thBlock_embeddings_df'


if not os.path.exists(os.path.join(PROJECT_DATA_PATH,FILENAME+'.pkl')):
    # Load the pre-trained WideResNet50 model
    model = models.wide_resnet50_2(pretrained=True)
    # Modify the model to extract features from the third residual block
    model = torch.nn.Sequential(*(list(model.children())[:7]))
    # Function to extract features

    def extract_features(img_path, model):
        # Load image with PIL and convert to RGB
        img = Image.open(img_path).convert("RGB")

        # Apply transformations
        transform = transforms.Compose([
            transforms.Resize([224, 224]),#resize for wideresent
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) #normalize  images for imagenet
        ])

        img = transform(img).unsqueeze(0)  # Add batch dimension

        with torch.no_grad():
            features = model(img)
        return features[0].flatten()#.squeeze().numpy()


    all_embeddings = []
    # Extract features and save to DataFrame
    for img_path in image_paths:
        batch_embeddings = extract_features(img_path, model)
        all_embeddings.append(batch_embeddings)
    tensor_list_2d = [t.unsqueeze(0) for t in all_embeddings]
    all_embeddings = torch.cat(tensor_list_2d, dim=0)
    
    df = pd.DataFrame(all_embeddings, index=image_paths)
    df.to_pickle(os.path.join(PROJECT_DATA_PATH, FILENAME + '.pkl'))
    
else:
    df = pd.read_pickle(os.path.join(PROJECT_DATA_PATH,FILENAME+'.pkl'))
    
df.head()